In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import math
import nltk
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re

In [2]:
# !pip install -U transformers
#!pip install -U accelerate

#!pip install datasets
#!pip install evaluate

### Creando base de datos de sentimientos

https://github.com/garnachod/TwitterSentimentDataset

In [3]:
neu = pd.read_csv("TwitterSentimentDataset/tweets_clean.txt", sep="\t",header=None, names=["frase"])
neu['label'] = 0

In [4]:
neu.shape

(70194, 2)

In [5]:
neu.sample(10).frase.tolist()

['hoy decido ser feliz y no andar dándole importancia a pendejadas!  para eso no se hizo la vida, tengan un bonito domingo! :D',
 'Los creadores de Torchlight presentan Hob, una aventura de acción y puzles para PC y consolas http://t.co/XIbgYyBJof',
 '@HdLG94 por algo en particular o el también lo lleva sospechando de hace tiempo?',
 '@AdvyStyles Guitaaaar llevo días pidiéndote esto D: Juega este Demon pls 10875730 Lo hice con mucho esfuerzo :D',
 '#PreguntaPisito #AlexbyManco Aprende a jugar ?:D',
 'Ahora mismo @Irene_Montero_ en @noticias_cuatro en el especial #GreciaSeLaJuegaN4',
 'Jajaja :p',
 'Holu seguir is te sigo soi nueva :D https://t.co/7cmPJqvvZK',
 '@Cernizo Profe profe que se está copiando',
 'Exigimos la renovación del sistema de financiación autonómica y un reparto más equilibrado de los objetivos de déficit @sanchezcastejon']

In [6]:
neg = pd.read_csv("TwitterSentimentDataset/tweets_neg_clean.txt", sep="\t",header=None, names=["frase"])
neg['label'] = 1

In [7]:
neg.shape

(120948, 2)

In [8]:
neg.sample(10).frase.tolist()

['Si me lo dice ella,  si importante :(',
 'opioo la extraño :( #okno :3',
 'ya me urge escáparme al D.F. en septiembre, necesito cambiar de aires unos días :(',
 'Y es todo tan dificil , y toda cuesta tanto #cabezaamil #quierolodeantes :( •',
 '@TheTideDrea @ashswonderwall ya hija ya, es el puto cody que va en esas fechas :(( pero lo bueno es que me quedo una semana por ahí :p',
 'de Delta me mandaron a comer al Gatsby e increíblemente la comida era lo bueno y los postres todos malos y añejos :(',
 'Quiero a mi novio conmigo :(',
 '@LaddySally Antes hemos estado invitando a uno pero no le gustaba la idea que teníamos planeada para él :(',
 'Como cuando te quieres tatuar pero no tienes un buen lienzo jajajajajajajajajaajqjqjjq ay :(',
 'se me re pasparon los labios del frio hoy :(']

In [9]:
pos = pd.read_csv("TwitterSentimentDataset/tweets_pos_clean.txt", sep="\t",header=None, names=["frase"])
pos['label'] = 2

In [10]:
pos.shape

(55056, 2)

In [11]:
pos.sample(10).frase.tolist()

['Domingo de Paz... #picadita #Asado #Campo #familiaa #Cuñaditoo. :)',
 '@caromiyares estamos creando un listado de salas y garitos donde paguen a los músicos, nos ayudas con un RT? :) http://t.co/UcqIigCDbQ',
 'Ahora si ! NUNCA MÁS :)',
 '@maxekampos era bueno el juego de palabras pero lamentablemente esta vez no aplicó ;)',
 '@ExtasisTextual Gracias por seguirme, en breve te devuelvo follow :) #TuitUtil http://t.co/wyrm2Ck0sR',
 '@JamesTaiss Gracias por seguirme, en breve te devuelvo follow :) #TuitUtil http://t.co/RnR1hpgjFn',
 '@dmarquina ya mandaré fotillos. :)',
 '@RHPlusConsultor @CarlitaAlboran @MyrnaBarajas Buenas :), Quieres ser único en tu regalo, desayunos a domicilio http://t.co/OO7vuR4kRo',
 '@natipaesky Hola, estoy participando de un concurso con una foto de MIRAMAR. Me ayudas con un MG? :) https://t.co/sX97RTq0DM',
 '@Daniela_PerezM jajaja de nada :)']

In [12]:
sent = pd.concat([pos.sample(500),neg.sample(500),neu.sample(500)])

In [13]:
sent

,frase,label
7867,Amén :) http://t.co/DtqheyTxXW,2
51008,¡Ya podéis visitar la web de @InframundoEdit e...,2
18094,Con postre de chocolate y vino... :) http://t....,2
12656,El Me Dice Ke Me Ama.. i Me Lo Demuetra CoN Ec...,2
34481,Llegar a casa y encontrar estas joyas en la pu...,2
...,...,...
58869,¿Te han preguntado alguna vez cómo ir a las el...,0
15287,@Cami_Sediari Me haces un favor? Podés leer el...,0
44032,Juego de maquillar a Clawdeen de Monster High:...,0
38689,"Qué pasará, qué misterios habrá, puede ser mi ...",0


In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sent, random_state=42, test_size=0.25)

In [15]:
train.shape

(1125, 2)

In [16]:
train.sample(10)

,frase,label
60071,".@Rafa_Hernando: ""El PP sitúa a las personas e...",0
113298,@Malaspinitasgh somos dos! Era increíble escuc...,1
79429,"No tengo crushes como desde hace 5 meses, por ...",1
17175,"#QuiénNoHaGritado Ahora no, Ma!",0
45782,@JosephinSkrver Yo mucho mucho mucho más :(,1
17661,Mi mama postiza miri se la juega mañana :D,0
53793,Comenzamos muy bien el domingo :) #TacosJalisc...,2
31288,Me ha gustado un vídeo de @YouTube de @brooxix...,0
50569,@combatearg LLOROO :-(:-(:'(:'(:'(,1
65501,@PedroPepa_ no puedo :( tuve que dejar morir e...,1


Finalmente, vamos a eliminar las caritas tristes y felices. De no hacerlo, pasan cosas curiosas...

In [17]:
train['frase'] = train['frase'].str.replace(':)','')
train['frase'] = train['frase'].str.replace(':(','')
test['frase'] = test['frase'].str.replace(':)','')
test['frase'] = test['frase'].str.replace(':(','')

### BERT 

### Fine tuning for classification

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [19]:
from datasets import Dataset   # para transformar la data al formato Huggingface
import evaluate   # para importar las métricas de evaluación del modelo

Cargando el modelo pre-trained BERT y el tokenizador. En este caso, Roberta, porque BETO no funcionó. 

https://huggingface.co/MMG/xlm-roberta-base-sa-spanish

In [20]:
tokenizer = AutoTokenizer.from_pretrained("MMG/xlm-roberta-base-sa-spanish")

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("MMG/xlm-roberta-base-sa-spanish",num_labels=3)

In [22]:
tokenizer

XLMRobertaTokenizerFast(name_or_path='MMG/xlm-roberta-base-sa-spanish', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [23]:
tokenizer.sep_token

'</s>'

In [24]:
tokenizer.sep_token_id

2

In [25]:
list(tokenizer.vocab.keys())[0:10]

['▁облека',
 'తు',
 '▁pemberitahuan',
 '▁travel',
 '▁colonia',
 '▁vortoj',
 '၇',
 'imbaji',
 'rance',
 '▁faks']

In [26]:
len(list(tokenizer.vocab.keys()))

250002

Conviertiendo la data en algo que HuggingFace pueda leer. La gracia de los objetos tipo HG Dataset es que son memory-mapped, esto es, que el contenido se almacena en una memoria virtual y por lo mismo, no está sujeta a la RAM del compu de uno. 

In [27]:
hg_train_data = Dataset.from_pandas(train)
hg_test_data = Dataset.from_pandas(test)

In [28]:
hg_train_data[0]

{'frase': 'Oficialmente te odio tarea  #MTVHottest Justin Bieber',
 'label': 1,
 '__index_level_0__': 85523}

In [29]:
len(hg_train_data)

1125

Como vemos, cada elemento es un diccionario, que hereda los nombre de los campos de acuerdo a las columnas del dataframe, y añade un campo que corresponde al índice. Por lo demás, tiene el mismo largo que el dataframe original.

Para aplicar el tokenizador a todo el dataset, lo definimos dentro de una función, para luego poder mapearla. Tenemos que definir cual es el número de máximo de tokens para los documentos, sobre eso se su truncará y en base a eso se aplicará el padding.

In [30]:
# Tokenize the texts
def tokenize_dataset(data):
 return tokenizer(data['frase'], max_length= 32, padding='max_length', truncation=True)   # return_tensors="pt"

In [31]:
dataset_train = hg_train_data.map(tokenize_dataset,batched=True)
dataset_test = hg_test_data.map(tokenize_dataset,batched=True)

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [32]:
print(dataset_train)

Dataset({
    features: ['frase', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1125
})


In [33]:
#dataset_train[2]

El paso siguiente es especificar los argumentos con que se entrenará el modelo. Los parámetros del Trainer son muchos (del orden de 100), lo que da mucha flexibilidad (para el que quiera meterse ahí). Algunos que me parecieron importantes:

eval strategy <br>
num_train_epochs  <br>
output_dir <br>
per_device_train_batch_size <br>

Un epoch consiste en un ciclo completo del dataset de entranamiento, y se compone de varios steps. Cuantos, depende del batch size. <br>
Number of Steps per Epoch = (Total Number of Training Samples) / (Batch Size)

- The batch size is a hyperparameter of gradient descent that controls the number of training samples to work through before the model’s internal parameters are updated.
- The number of epochs is a hyperparameter of gradient descent that controls the number of complete passes through the training dataset.

https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArgument<br>
https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/s

In [34]:
# (1125/8)  = 141
# 141 * 3 = 423

In [35]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", num_train_epochs=3, per_device_train_batch_size=8)

In [36]:
dataset_test

Dataset({
    features: ['frase', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 375
})

In [37]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics = compute_metrics,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.834877,0.640000
2,No log,0.718271,0.677333


In [ ]:
trainer.evaluate(dataset_test)

In [ ]:
trainer.save_model('seq_classification2')

In [ ]:
y_test_predict = trainer.predict(dataset_test)

In [ ]:
y_test_predict.predictions

In [ ]:
y_test_predict.label_ids